Evan Callia, Kyle Kusuda, Rachel Roberts

This scraper scrapes the epa website containing a list of all the superfund zones in the United States. It grabs all relating information such as the name, id, indicator, etc. and then goes into the link it refrences in order to grab the address. Once all of the attribute data is grabbed and the location geocoded through nominatim, the list of superfund sites is output as a shapefile.

In [ ]:
#setting system pathways for arcmap
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts') 

#importhing methods used for scraping
import urllib2
import lxml.html
import requests
import json
import arcpy
arcpy.env.workspace = r"V:\UW\Geog 458\Lab03"
arcpy.env.overwriteOutput = True

#importing method used to scrape javascript rendered website
from selenium import webdriver
 
driver = webdriver.Firefox()
url = "http://www.epa.gov/superfund/national-priorities-list-npl-sites-state#WA"
driver.get(url)

dom = lxml.html.fromstring(driver.page_source)
dom.make_links_absolute(url)

#grab all places mentioned on website
rows = dom.cssselect("table tbody tr")

#this will be the list of dictionaries containg each site and all of it's attributes
#each dictionary will contain these elements: name, id, date, score, 
#indicator, link, city, state, and zipcode
sites = []

#keeps track of how many sites fail to geocode
count = 0

#go through each site and pull out attributes
for row in rows:
    #grab td elements (each containing one attribute)
    atts = row.cssselect("td")
    name = atts[0].text
    sid = atts[2].text
    date = atts[3].text
    score = atts[4].text
    indic = atts[5].text
    link = ""
    address = ""
            
    #grab appropriate link
    links = atts[6].cssselect("a")
    for templink in links:
        if templink.text == "Site Progress Profile":
            link = templink.attrib['href']

            #go into link and pull out address components            
            htmltwo = urllib2.urlopen(link).read()
            domtwo = lxml.html.fromstring(htmltwo)
            domtwo.make_links_absolute(link)
        
            comps = domtwo.cssselect("div#myrightbox br")
            if len(comps) > 4:
                if comps[2].tail is not None:
                    comp1 = comps[2].tail.strip()
                else:
                    comp1 = ""
                if comps[3].tail is not None:
                    comp2 = comps[3].tail.strip()
                else:
                    comp2 = ""
                if comps[4].tail is not None:
                    comp3 = comps[4].tail.strip()
                else:
                    comp3 = ""

                temp = []
                if "," in comp1:
                    temp = comp1.split(' ')
                elif "," in comp2:
                    temp = comp2.split(' ')
                else:
                    temp = comp3.split(' ')

                if len(temp) > 2:
                    city = temp[len(temp)-3].strip() + " "
                    state = temp[len(temp)-2].strip() + " "
                    zipcode = temp[len(temp)-1].strip()

                    sites.append({"name": name, "site_id": sid, "date": date, "score": score, "indicator": indic, "link": link, "city": city, "state": state, "zipcode": zipcode})
                else:
                    count += 1
            else:
                count += 1


In [ ]:
#list of our points to be made (populated later)
points = []

for site in sites:    
    #concatinate url searches and retrieve data in json format
    query = "http://nominatim.openstreetmap.org/search?q=" + site["city"].replace(" ", "+") + site["state"].replace(" ", "+") + site["zipcode"].replace(" ", "+") + "&format=json&polygon=1&addressdetails=1"
    response = requests.get(query)
    response_list = json.loads(response.content)
    
    #grab latitude and longitude from json returned
    lat = ""
    lon = ""
    for row2 in response_list:
        if "lat" in row2:
            lat = row2["lat"].strip()
        if "lon" in row2:
            lon = row2["lon"].strip()
            break
        
    #if score is invalid set to 9999
    if site["score"] is None:
        site["score"] = 9999
    
    #put the attributes and coordinates of each point in our points list
    #check for exception and skip point if so (point failed to geocode)
    try:
        points.append((site["name"], site["site_id"], site["date"], site["score"], site["indicator"], site["link"], site["city"], site["state"], site["zipcode"], [float(lon), float(lat)]))
    except ValueError,e:
        #print site["name"] + " Failed to geocode."
        count += 1
                     


In [ ]:
#make feature class to put our points in (setting refrecences and adding fields needed)
sr = arcpy.SpatialReference(4326)
arcpy.CreateFeatureclass_management(r"V:\UW\Geog 458\458 Final Project\Output", "superfund_sites.shp", "POINT","", "", "", sr)
for key in sites[0]:
    if key == "date":
        arcpy.AddField_management(r"V:\UW\Geog 458\458 Final Project\Output\superfund_sites.shp", str(key), "DATE", "", "", "", "", "NULLABLE")
    elif key == "score":
        arcpy.AddField_management(r"V:\UW\Geog 458\458 Final Project\Output\superfund_sites.shp", str(key), "DOUBLE", "", "", "", "", "NULLABLE")
    else:
        arcpy.AddField_management(r"V:\UW\Geog 458\458 Final Project\Output\superfund_sites.shp", str(key), "TEXT", "", "", "", "", "NULLABLE")
                     
#populate shapefile with our points
cursor = arcpy.da.InsertCursor(r"V:\UW\Geog 458\458 Final Project\Output\superfund_sites.shp", ("name", "site_id", "date", "score", "indicator", "link", "city", "state", "zipcode", "SHAPE@XY"))
for point in points:
    cursor.insertRow(point)
del cursor

print str(count) + " sites failed to geocode."



    